In [1]:
from scipy.optimize import minimize
import numpy as np
from random import gauss
import scipy as sp

In [2]:
def center_image_fiducials(dimensions,fiducials):

    ''' Centers image fiducial points using image dimensions  '''

    for i in range(np.shape(fiducials)[0]):
        fiducials[i,0] = fiducials[i,0] - (dimensions[0] / 2)
        fiducials[i,1] = (dimensions[1] / 2) - fiducials[i,1]

    return fiducials

In [3]:
def colin(params, xyz_a):

	# Unwrap params
	kappa, phi, omega, xs, ys, zs, f = params

	omega = float(omega)
	phi = float(phi) + 0.5*np.pi
	kappa = float(kappa)
	xs = float(xs)
	ys = float(ys)
	zs = float(zs)
	f = float(f)

	# -- utils
	co = np.cos(omega)
	so = np.sin(omega)
	cp = np.cos(phi)
	sp = np.sin(phi)
	ck = np.cos(kappa)
	sk = np.sin(kappa)

	a1 =  cp*ck+sp*so*sk
	b1 =  cp*sk+sp*so*ck
	c1 =  sp*co
	a2 = -co*sk
	b2 =  co*ck
	c2 =  so
	a3 =  sp*ck+cp*so*sk
	b3 =  sp*sk-cp*so*ck
	c3 =  cp*co

	ynum  = a1*(xyz_a[:,0]-xs)+b1*(xyz_a[:,1]-ys)+c1*(xyz_a[:,2]-zs)
	xnum  = a2*(xyz_a[:,0]-xs)+b2*(xyz_a[:,1]-ys)+c2*(xyz_a[:,2]-zs)
	denom = a3*(xyz_a[:,0]-xs)+b3*(xyz_a[:,1]-ys)+c3*(xyz_a[:,2]-zs)

	xx = -f*xnum/denom
	yy = f*ynum/denom

	return np.vstack([xx,yy]).T

        '''	
	# Get number of fiducial points
	N = int(xyz_a.shape[0])

	# Initialize the result vector
	colin_xy = np.zeros((N,2))

	# Define c code that will evaluate the functions
	code = """

		double sinp = sinf(phi);
		double cosp = cosf(phi);
		double sino = sinf(omega);
		double coso = cosf(omega);
		double sink = sinf(kappa);
		double cosk = cosf(kappa);

		double a1 = cosp*cosk;
		double b1 = coso*sink + sino*sinp*cosk;
		double c1 = sino*sink - coso*sinp*cosk;
		double a2 = -1 * cosp*sink;
		double b2 = coso*cosk - sino*sinp*sink;
		double c2 = sino*cosk + coso*sinp*sink;
		double a3 = sinp;
		double b3 = -1*sino*cosp;
		double c3 = cosp*coso;

		int j;
		int k;
		double denom;

		for(int i = 0; i < N; i++)
		{
			j = i*2;
			k = i*3;
			denom = (a3*(xyz_a[k]-xs) + b3*(xyz_a[(k+1)]-ys) + c3*(xyz_a[(k+2)]-zs));

			colin_xy[j] = -1.0*f*(a1*(xyz_a[k]-xs) + b1*(xyz_a[(k+1)]-ys) + c1*(xyz_a[(k+2)]-zs))/denom;
			colin_xy[(j+1)] = -1.0*f*(a2*(xyz_a[k]-xs) + b2*(xyz_a[(k+1)]-ys) + c2*(xyz_a[(k+2)]-zs))/denom;
		}
		return_val = 1;
	"""

	# Use scipy.weave.inline to run the c code
	res = inline(code, ['colin_xy', 'omega', 'phi', 'kappa', 'xs', \
		'ys', 'zs', 'f', 'xyz_a', 'N'], headers = ['<math.h>'], \
		compiler = 'gcc')
	
	# Return the pixel (x,y) positions
	return colin_xy
	'''

In [21]:
def colinN(params, xyz_a):

	# Unwrap params
	kappa, phi, omega, xs, ys, zs, f = params

	omega = float(omega)*2*np.pi
	phi = (float(phi)*0.5*np.pi)+0.5*np.pi
	kappa = float(kappa)*2*np.pi
	xs = (float(xs)*25000)+975000
	ys = (float(ys)*30000) + 192000
	zs = float(zs)*2000
	f = float(f)*3000

	# -- utils
	co = np.cos(omega)
	so = np.sin(omega)
	cp = np.cos(phi)
	sp = np.sin(phi)
	ck = np.cos(kappa)
	sk = np.sin(kappa)

	a1 =  cp*ck+sp*so*sk
	b1 =  cp*sk+sp*so*ck
	c1 =  sp*co
	a2 = -co*sk
	b2 =  co*ck
	c2 =  so
	a3 =  sp*ck+cp*so*sk
	b3 =  sp*sk-cp*so*ck
	c3 =  cp*co

	ynum  = a1*(xyz_a[:,0]-xs)+b1*(xyz_a[:,1]-ys)+c1*(xyz_a[:,2]-zs)
	xnum  = a2*(xyz_a[:,0]-xs)+b2*(xyz_a[:,1]-ys)+c2*(xyz_a[:,2]-zs)
	denom = a3*(xyz_a[:,0]-xs)+b3*(xyz_a[:,1]-ys)+c3*(xyz_a[:,2]-zs)

	xx = -f*xnum/denom
	yy = f*ynum/denom

	return np.vstack([xx,yy]).T

        '''	
	# Get number of fiducial points
	N = int(xyz_a.shape[0])

	# Initialize the result vector
	colin_xy = np.zeros((N,2))

	# Define c code that will evaluate the functions
	code = """

		double sinp = sinf(phi);
		double cosp = cosf(phi);
		double sino = sinf(omega);
		double coso = cosf(omega);
		double sink = sinf(kappa);
		double cosk = cosf(kappa);

		double a1 = cosp*cosk;
		double b1 = coso*sink + sino*sinp*cosk;
		double c1 = sino*sink - coso*sinp*cosk;
		double a2 = -1 * cosp*sink;
		double b2 = coso*cosk - sino*sinp*sink;
		double c2 = sino*cosk + coso*sinp*sink;
		double a3 = sinp;
		double b3 = -1*sino*cosp;
		double c3 = cosp*coso;

		int j;
		int k;
		double denom;

		for(int i = 0; i < N; i++)
		{
			j = i*2;
			k = i*3;
			denom = (a3*(xyz_a[k]-xs) + b3*(xyz_a[(k+1)]-ys) + c3*(xyz_a[(k+2)]-zs));

			colin_xy[j] = -1.0*f*(a1*(xyz_a[k]-xs) + b1*(xyz_a[(k+1)]-ys) + c1*(xyz_a[(k+2)]-zs))/denom;
			colin_xy[(j+1)] = -1.0*f*(a2*(xyz_a[k]-xs) + b2*(xyz_a[(k+1)]-ys) + c2*(xyz_a[(k+2)]-zs))/denom;
		}
		return_val = 1;
	"""

	# Use scipy.weave.inline to run the c code
	res = inline(code, ['colin_xy', 'omega', 'phi', 'kappa', 'xs', \
		'ys', 'zs', 'f', 'xyz_a', 'N'], headers = ['<math.h>'], \
		compiler = 'gcc')
	
	# Return the pixel (x,y) positions
	return colin_xy
	'''

In [36]:
def fullfunc(params,xyz_s,xy_t):

    ''' Find the sum of squares difference '''
    omega, phi, kappa, xs, ys, zs, f = params


#     if (omega<0.0) or (omega>=2.0*np.pi):
#         return 1e9+omega**4
#     elif (phi<-0.5*np.pi) or (phi>=0.5*np.pi):
#         return 1e9+phi**4
#     elif (kappa<0.0) or (kappa>=2.0*np.pi):
#         return 1e9+kappa**4
#     elif zs<0.0:
#         return 1e9+zs**4
#     elif f<0.0:
#         return 1e9+f**4

#     elif (np.abs(params[3] - 977119)>1000) or \
#             (np.abs(params[4] - 210445)>1000):
#         return 1e9 + xs**2

    colin_xy = 1.0*colinN(params,xyz_s.astype(float))
    diff = ((colin_xy - xy_t)**2).sum()

    return diff

In [32]:
def call(params,xyz_s,xy_t):

    ''' Guess parameters near start and brute-force minimize '''
    start = params
    
    boundsN=[(0.01, 0.99), (-0.99, 0.99), (0.01, 0.99),
            (0.0001, 0.9999), (0.0001, 0.9999), (0.35, 0.45), (0.1, 1.2)]
    
    res = minimize(fullfunc, start,args=(xyz_s,xy_t), \
        method = 'L-BFGS-B', \
        #options={'maxfev': 10000, 'maxiter': 10000})    # Nelder-Mead
        options={'disp': True, 'maxiter': 10000, 'gtol': 0.001}, bounds=boundsN) #BFGS

    return res

In [8]:
def random_start(params):

# 	''' Perturbs camera position in a gaussian fashion '''
# 	return params
# 	return params + np.array([gauss(0, 0.1), gauss(0, 0.1), \
# #	gauss(0, 0.1),gauss(0, 20000),gauss(0, 20000), \
# 	gauss(0, 0.1),gauss(0, 100),gauss(0, 100), \
# 	gauss(0, 100), gauss(0, 30)])
# #	gauss(0, 10), gauss(0, 2000)])

    return params + np.array([gauss(0, 0.1), gauss(0, 0.1), gauss(0, 0.1),
                              gauss(0, 0.1), gauss(0, 0.1),
                              gauss(0, 0.1), gauss(0, 0.1)])

In [9]:
lidar_fiducials = np.array([
                            [988328.741, 211962.778,1020.623838], #Empire state building
                            [980648.447, 199131.861,1380.707975], #WTC
                            [987656.616, 211766.233,493.89], # 1250 Broadway
                            [983564.98, 199358.775,591.406796], # Marshall courthouse
                            [987342.468, 212511.054,380.69], #  112 West 34th St
                            [988596.086, 211789.785,255.31], # 347 5th Ave
                            [988287.232, 213228.734,488.716947],  # 66 W 38th St
                            [984574.798, 204219.63, 303.263412], # 505 LaGuardia Place, second leftmost edge 
                            [987140.31, 209161.591, 324.317488], # Flatiron, left point on triange rooftop, under the fence
                            [986252.724, 205516.404, 365.468789]]) # 755 Broadway, leftmost corner on the roof
                            

In [37]:
xyz_s = lidar_fiducials

In [38]:
fiducials = np.array([
                        [507, 856],#Empire state building
                        [1681, 1004],# WTC
                        [1185, 1400], # 1250 Broadway
                        [1217, 1143], # Marshall courthouse
                        [1860, 1637], #  112 West 34th St
                        [211, 1704], # 347 5th Ave
                        [814, 1811], # 66 W 38th St
                        [1362, 1234], # 505 LaGuardia Place
                        [1037, 1355], # Flatiron
                        [1013, 1237]]) # 755 Broadway

dimensions = np.array([2560, 1918])
fiducials = center_image_fiducials(dimensions,fiducials)

In [40]:
xy_t = fiducials

In [42]:
(988491.-975000)/25000

0.53964

In [45]:
(214475. - 192000)/30000

0.7491666666666666

In [48]:
800./2000

0.4

In [49]:
1000./3000

0.3333333333333333

In [41]:
guessN = np.array([0.75, -0.2, 0, 0.53964, 0.7491666666666666, 0.4, 0.3333])
#guess = np.array([4.48603184, -5.75616093e-02, 0.0115, 988491, 214475, 830, 1000])
min_score = 100000000000000
num_iter = 100
params = guessN
for i in range(0, num_iter):
    start = random_start(params)
    result = call(start,xyz_s,xy_t)
    print "params: ", result.x
    print "score: ", result.fun
    if (result.fun < min_score):# and (result.x[3] < 980491):
        min_score = result.fun
        params = result.x

params:  [ 0.63725499  0.52588546  0.01        0.42960859  0.41830942  0.45
  0.1203711 ]
score:  3003527.49976
params:  [ 0.4135676   0.19657533  0.38644949  0.5677964   0.29963699  0.45        0.1       ]
score:  2189758.2703
params:  [ 0.41356306  0.19650606  0.3864337   0.56774665  0.2996375   0.45        0.1       ]
score:  2189758.26423
params:  [ 0.4135639   0.19651161  0.38643577  0.56775325  0.29963331  0.45        0.1       ]
score:  2189758.26391
params:  [ 0.241461    0.49738704  0.29951742  0.28146491  0.28801361  0.43321059
  0.19132298]
score:  2505137.74926
params:  [ 0.41356364  0.19651406  0.38643632  0.5677547   0.29962709  0.45        0.1       ]
score:  2189758.26431
params:  [ 0.71943095  0.54206355  0.24576627  0.96066607  0.91843774  0.45        0.1       ]
score:  2570433.73899
params:  [ 0.4135619   0.19651315  0.38643604  0.56774771  0.29963107  0.45        0.1       ]
score:  2189758.26409
params:  [ 0.74249177  0.38493108  0.55564381  0.44153148  0.55984434

In [42]:
print params, min_score

[ 0.69640964  0.00666957  0.99        0.54138304  0.74986585  0.40911916
  0.94887299] 8195.82946613


In [19]:
def params_orig(params):
    params_orig = [
        params[0]*2*np.pi,
        params[1]*0.5*np.pi,
        params[2]*2*np.pi,
        (params[3]*25000)+975000,
        (params[4]*30000) + 192000,
        params[5]*2000,
        params[6]*3000

    ]
    return params_orig

In [29]:
params_orig(params)

[2.8006137726991773,
 1.5773565325573278,
 -4.6622863564207586,
 988515.6299398191,
 214506.4613172313,
 802.36941702193383,
 2836.4987635824887]